# LSTM models on the UCI datasets
- LSTM Models are slow to train. The newly proposed TCNs are shown to be simpler and easier to train, along with better performance

In [7]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [8]:
from tensorflow.keras.utils import to_categorical

# Data loading utilities

In [9]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [10]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


# Using LSTM

In [11]:
# LSTM Network Default input: [batch, timesteps, feature]
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.LSTM(8, activation='relu'))
#model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8)                 576       
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 6)                 54        
Total params: 630
Trainable params: 630
Non-trainable params: 0
_________________________________________________________________


In [13]:
filepath = 'LSTM_Best_simple.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=50,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/50
7296/7352 [============================>.] - ETA: 0s - loss: 1.4130 - acc: 0.5256
Epoch 00001: val_acc improved from 0.47031 to 0.54123, saving model to LSTM_Best_simple.hdf5
7352/7352 [==============================] - 7s 906us/sample - loss: 1.4124 - acc: 0.5264 - val_loss: 1.4374 - val_acc: 0.5412
Epoch 2/50
7296/7352 [============================>.] - ETA: 0s - loss: 1.4086 - acc: 0.5387
Epoch 00002: val_acc did not improve from 0.54123
7352/7352 [==============================] - 8s 1ms/sample - loss: 1.4072 - acc: 0.5392 - val_loss: 1.4901 - val_acc: 0.5273
Epoch 3/50
7296/7352 [============================>.] - ETA: 0s - loss: 1.2593 - acc: 0.5785
Epoch 00003: val_acc improved from 0.54123 to 0.54360, saving model to LSTM_Best_simple.hdf5
7352/7352 [==============================] - 8s 1ms/sample - loss: 1.2594 - acc: 0.5785 - val_loss: 1.3757 - val_acc: 0.5436
Epoch 4/50
7296/7352 [============================>.] - ETA:

Epoch 31/50
7296/7352 [============================>.] - ETA: 0s - loss: 0.9408 - acc: 0.6187
Epoch 00031: val_acc did not improve from 0.63963
7352/7352 [==============================] - 9s 1ms/sample - loss: 0.9410 - acc: 0.6185 - val_loss: 1.1167 - val_acc: 0.6152
Epoch 32/50
7296/7352 [============================>.] - ETA: 0s - loss: 0.9428 - acc: 0.6154
Epoch 00032: val_acc did not improve from 0.63963
7352/7352 [==============================] - 11s 1ms/sample - loss: 0.9417 - acc: 0.6164 - val_loss: 1.1153 - val_acc: 0.6200
Epoch 33/50
7296/7352 [============================>.] - ETA: 0s - loss: 0.9391 - acc: 0.6235
Epoch 00033: val_acc did not improve from 0.63963
7352/7352 [==============================] - 11s 1ms/sample - loss: 0.9401 - acc: 0.6228 - val_loss: 1.1161 - val_acc: 0.6172
Epoch 34/50
7296/7352 [============================>.] - ETA: 0s - loss: 0.9425 - acc: 0.6180
Epoch 00034: val_acc did not improve from 0.63963
7352/7352 [==============================] - 11

In [14]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("LSTM model, accuracy: {:5.2f}%".format(100 * acc))

2947/2947 - 1s - loss: 1.1632 - acc: 0.6396
LSTM model, accuracy: 63.96%


# A GRU Layer of Keras

In [23]:
# LSTM Network Default input: [batch, timesteps, feature]
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.GRU(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 16)                1248      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
Total params: 1,622
Trainable params: 1,622
Non-trainable params: 0
_________________________________________________________________


In [25]:
filepath = 'GRU_Best_simple.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=128*4, epochs=40,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/40
7168/7352 [============================>.] - ETA: 0s - loss: 1.3140 - acc: 0.4867
Epoch 00001: val_acc improved from -inf to 0.45945, saving model to GRU_Best_simple.hdf5
7352/7352 [==============================] - 2s 267us/sample - loss: 1.3116 - acc: 0.4880 - val_loss: 1.3217 - val_acc: 0.4595
Epoch 2/40
7168/7352 [============================>.] - ETA: 0s - loss: 1.2691 - acc: 0.4968
Epoch 00002: val_acc improved from 0.45945 to 0.48660, saving model to GRU_Best_simple.hdf5
7352/7352 [==============================] - 2s 268us/sample - loss: 1.2668 - acc: 0.4977 - val_loss: 1.2774 - val_acc: 0.4866
Epoch 3/40
7168/7352 [============================>.] - ETA: 0s - loss: 1.1979 - acc: 0.5539
Epoch 00003: val_acc improved from 0.48660 to 0.57448, saving model to GRU_Best_simple.hdf5
7352/7352 [==============================] - 2s 276us/sample - loss: 1.1963 - acc: 0.5552 - val_loss: 1.1931 - val_acc: 0.5745
Epoch 4/40
7168/735

7168/7352 [============================>.] - ETA: 0s - loss: 0.2720 - acc: 0.8926
Epoch 00028: val_acc did not improve from 0.82660
7352/7352 [==============================] - 3s 379us/sample - loss: 0.2729 - acc: 0.8927 - val_loss: 0.4856 - val_acc: 0.8225
Epoch 29/40
7168/7352 [============================>.] - ETA: 0s - loss: 0.2715 - acc: 0.8934
Epoch 00029: val_acc did not improve from 0.82660
7352/7352 [==============================] - 3s 366us/sample - loss: 0.2720 - acc: 0.8930 - val_loss: 0.4579 - val_acc: 0.8229
Epoch 30/40
7168/7352 [============================>.] - ETA: 0s - loss: 0.2636 - acc: 0.8945
Epoch 00030: val_acc improved from 0.82660 to 0.83068, saving model to GRU_Best_simple.hdf5
7352/7352 [==============================] - 3s 374us/sample - loss: 0.2619 - acc: 0.8955 - val_loss: 0.5271 - val_acc: 0.8307
Epoch 31/40
7168/7352 [============================>.] - ETA: 0s - loss: 0.3060 - acc: 0.8771
Epoch 00031: val_acc improved from 0.83068 to 0.84086, saving m

In [26]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("GRU model, accuracy: {:5.2f}%".format(100 * acc))

2947/2947 - 2s - loss: 0.4227 - acc: 0.8575
GRU model, accuracy: 85.75%
